In [8]:
# https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html 
# Install required libraries
# Import required libraries
import random
import math
import gymnasium as gym
from gymnasium import spaces
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple, deque
from itertools import count

device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"
)

print(device)
CUDA_LAUNCH_BLOCKING=1

cuda


In [9]:
# imports are always needed
import torch
# get index of currently selected device
torch.cuda.current_device() # returns 0 in my case
# get number of GPUs available
torch.cuda.device_count() # returns 1 in my case
# get the name of the device
torch.cuda.get_device_name(0) # good old Tesla K80


'NVIDIA GeForce GTX 1660 SUPER'

In [15]:

# modify this to fit current environment 
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
class DQN(nn.Module):

    def __init__(self, obs, action):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(obs, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, action)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)
    
class agent():
    def __init__(self,obs,actions):
        self.actions = actions
        self.batch = 128
        self.discount_factor = 0.99
        self.eps = 1
        self.eps_decay = 0.9954
        # self.tau = 0.005
        self.learning_rate = 0.1
        self.memory = ReplayMemory(5000) # replay memory 
        # self.policy_net = DQN(obs, actions).to(device) # action value function
        # self.target_net = DQN(obs, actions).to(device) # target action value function 
        self.policy_net = DQN(obs, actions)
        self.target_net = DQN(obs, actions)
        self.optimizer = optim.SGD(self.policy_net.parameters(), lr=0.01)
        self.Q_table = self.print_table() # not working rn
        
    
    def select_action(self,state):
        p = random.random()
        if p < self.eps:
            #random action
            return torch.tensor([[env.action_space.sample()]], dtype=torch.long)
        else:
            #best action
            with torch.no_grad():
                # t.max(1) will return the largest column value of each row.
                # second column on max result is index of where max element was
                # found, so we pick action with the larger expected reward.
                return self.policy_net(state).max(1).indices.view(1, 1)

    def update(self,timestep,episode,terminated):
        #update weight every 5 steps
        t = timestep +1
        if t%5 == 0:
            #sample and train policy_net
            self.replay()
        #update target_net every 5 eps, If solved, set as target_net
        ep = episode+1
        if ep+1%5 == 0 or terminated == True:
            self.target_net.load_state_dict(self.policy_net.state_dict())
        return 
    
    def replay(self):
        if len(agent.memory) < self.batch:
            return
        else:
            print("sample")
            sample = self.memory.sample(self.batch)
            q_values_list = []
            target_list = []
            for state, action, next_state, reward in sample:
                if next_state == None:
                    target = reward
                else:
                    with torch.no_grad():
                        #predict target Q-values
                        q = self.target_net(next_state).max(1)[0]
                        next_q = q.item() #predict next state with target network 
                        target = (reward + self.discount_factor * np.amax(next_q))
                q_values = self.policy_net(state)[0][action].item()#predict with policy network
                q_values_list.append(q_values)
                target_list.append(target)
            criterion = nn.MSELoss()
            state_action_list = torch.FloatTensor(q_values_list)
            target_list = torch.FloatTensor(target_list)
            print(q_values_list)
            loss = criterion(state_action_list,target_list.unsqueeze(1))
            print(loss)
            #backprop
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            return 
        #replay algorithim here
    def print_table(self): # not working rn
        
        return



env = gym.make("CartPole-v1")
state, info = env.reset()
obs = len(state)
actions = env.action_space.n
agent = agent(obs,actions)
        
    

[ 0.03632089  0.04400472 -0.04237615  0.02615733]


In [11]:
#create environment to run DQN
env = gym.make("CartPole-v1")

max_episodes = 600
max_timestep = 500
state, info = env.reset()
obs = len(state)
actions = env.action_space.n
# agent = agent(obs,actions)
reward_per_episode = []
for episode in range(max_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
    terminated,truncated = False, False
    for timestep in range(500):
        action = agent.select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward +=timestep
        if terminated:
            next_state = None #used to check if object has fallen during replay
        else:
            next_state = torch.tensor(observation, dtype=torch.float32).unsqueeze(0)

        agent.memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state
        agent.update(timestep=timestep,episode=episode,terminated=truncated) #update network weights
        if terminated or truncated: # close loop if it ends
            print('loop end at:', timestep)
            reward_per_episode.append(timestep)
            break
    agent.eps = agent.eps * agent.eps_decay #eps decay

print('solved',truncated)
plt.figure()
fig ,  ax = plt.subplots()
plt.xlabel('Episode', fontsize=20)
plt.ylabel('Timestep', fontsize=20)
plt.title('Cumulative Reward Per Episode', fontsize=24)
ax.plot(reward_per_episode,linestyle='solid',label = 'Q learning')
plt.grid()
plt.show()

loop end at: 30
loop end at: 20
loop end at: 21
loop end at: 31
sample
[0.17321006953716278, 0.15832164883613586, -0.12193575501441956, 0.17006728053092957, -0.11164073646068573, 0.16719035804271698, -0.12749534845352173, -0.13986122608184814, 0.1621609330177307, -0.09154041856527328, 0.15535110235214233, 0.171651691198349, -0.12102355062961578, -0.07319142669439316, 0.16889330744743347, 0.041560158133506775, -0.10240452736616135, 0.0056761205196380615, -0.10393916070461273, -0.10222607851028442, -0.10769146680831909, 0.16627781093120575, -0.11273528635501862, 0.15874060988426208, -0.0950411781668663, 0.16426166892051697, 0.23476621508598328, 0.16141393780708313, 0.17101672291755676, -0.11397510766983032, -0.10279591381549835, 0.17100752890110016, 0.16136735677719116, -0.09294819831848145, -0.061377912759780884, 0.17409339547157288, 0.16930729150772095, -0.11097672581672668, 0.1695980429649353, -0.08604257553815842, -0.10822220891714096, -0.12567773461341858, 0.16720831394195557, -0.09

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
target = torch.randn(10).view(1,-1)
print(target)